# zib gereed maken voo rmapping

* download van zibs
* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2020)

In [45]:
base = 'zibs2020'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['Nl.zorg.Alert-v4.1(2020EN).xlsx',
 'Nl.zorg.AllergyIntolerance-v3.3(2020EN).xlsx',
 'Nl.zorg.HealthcareProvider-v3.4(2020EN).xlsx',
 'Nl.zorg.HealthProfessional-v3.5(2020EN).xlsx',
 'Nl.zorg.MedicalDevice-v3.3.1(2020EN).xlsx',
 'Nl.zorg.Patient-v3.2(2020EN).xlsx',
 'Nl.zorg.Problem-v4.4(2020EN).xlsx',
 'Nl.zorg.Procedure-v5.2(2020EN).xlsx']

## kies een zib

In [52]:
file = 'Nl.zorg.Problem-v4.4(2020EN).xlsx'
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.Problem-v4.4(2020EN).xlsx


c:\Users\marcd\Anaconda3\envs\fhir\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,Problem,NaN,NaN,NaN,NaN,NaN,NL: Probleem,NaN,NaN,rootconcept,NL-CM:5.1.1,Root concept of the Problem information model....,NaN,NaN,NaN
1,NaN,NaN,ProblemType,NaN,NaN,NaN,NaN,NL: ProbleemType,CD,0..1,data,NL-CM:5.1.8,The type of problem; see the concept description.,NaN,ProblemTypeCodelist,NaN
2,NaN,NaN,ProblemName,NaN,NaN,NaN,NaN,NL: ProbleemNaam,CD,1,data,NL-CM:5.1.3,The problem name defines the problem. Dependin...,NaN,ProblemNameCodelist,NaN
3,NaN,NaN,FurtherSpecificationProblemName,NaN,NaN,NaN,NaN,NL: NadereSpecificatieProbleemNaam,ST,0..1,data,NL-CM:5.1.13,Further specification of problem name when it ...,NaN,NaN,NaN
4,NaN,NaN,ProblemAnatomicalLocation::AnatomicalLocation,NaN,NaN,NaN,NaN,NL: ProbleemAnatomischeLocatie::AnatomischeLoc...,NaN,0..1,"data,reference",NL-CM:5.1.14,Anatomical location which is the focus of the ...,SNOMED CT: 405813007 Procedure site - Direct,This is a reference to the rootconcept of info...,NaN
5,NaN,NaN,ProblemStartDate,NaN,NaN,NaN,NaN,NL: ProbleemBeginDatum,TS,0..1,data,NL-CM:5.1.6,"Onset of the symptom, complaint, functional li...",NaN,NaN,NaN
6,NaN,NaN,ProblemEndDate,NaN,NaN,NaN,NaN,NL: ProbleemEindDatum,TS,0..1,data,NL-CM:5.1.9,Date on which the disorder to which the proble...,NaN,NaN,NaN
7,NaN,NaN,ProblemStatus,NaN,NaN,NaN,NaN,NL: ProbleemStatus,CD,1,data,NL-CM:5.1.4,The problem status describes the condition of ...,NaN,ProblemStatusCodelist,NaN
8,NaN,NaN,VerificationStatus,NaN,NaN,NaN,NaN,NL: VerificatieStatus,CD,0..1,data,NL-CM:5.1.10,Clinical status of the problem or the diagnosis.,SNOMED CT: 408729009 Finding context,VerificationStatusCodelist,NaN
9,NaN,NaN,Comment,NaN,NaN,NaN,NaN,NL: Toelichting,ST,0..1,data,NL-CM:5.1.5,Comment by the one who determined or updated t...,LOINC: 48767-8 Annotation comment [Interpretat...,NaN,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)
* kolommen opschonen tot benodigde

In [53]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,Problem,Problem,NL: Probleem,,,rootconcept,NL-CM:5.1.1,Root concept of the Problem information model....,
1,ProblemType,Problem.ProblemType,NL: ProbleemType,CD,0..1,data,NL-CM:5.1.8,The type of problem; see the concept description.,
2,ProblemName,Problem.ProblemName,NL: ProbleemNaam,CD,1,data,NL-CM:5.1.3,The problem name defines the problem. Dependin...,
3,FurtherSpecificationProblemName,Problem.FurtherSpecificationProblemName,NL: NadereSpecificatieProbleemNaam,ST,0..1,data,NL-CM:5.1.13,Further specification of problem name when it ...,
4,ProblemAnatomicalLocation::AnatomicalLocation,Problem.ProblemAnatomicalLocation::AnatomicalL...,NL: ProbleemAnatomischeLocatie::AnatomischeLoc...,,0..1,"data,reference",NL-CM:5.1.14,Anatomical location which is the focus of the ...,SNOMED CT: 405813007 Procedure site - Direct
5,ProblemStartDate,Problem.ProblemStartDate,NL: ProbleemBeginDatum,TS,0..1,data,NL-CM:5.1.6,"Onset of the symptom, complaint, functional li...",
6,ProblemEndDate,Problem.ProblemEndDate,NL: ProbleemEindDatum,TS,0..1,data,NL-CM:5.1.9,Date on which the disorder to which the proble...,
7,ProblemStatus,Problem.ProblemStatus,NL: ProbleemStatus,CD,1,data,NL-CM:5.1.4,The problem status describes the condition of ...,
8,VerificationStatus,Problem.VerificationStatus,NL: VerificatieStatus,CD,0..1,data,NL-CM:5.1.10,Clinical status of the problem or the diagnosis.,SNOMED CT: 408729009 Finding context
9,Comment,Problem.Comment,NL: Toelichting,ST,0..1,data,NL-CM:5.1.5,Comment by the one who determined or updated t...,LOINC: 48767-8 Annotation comment [Interpretat...


## Excel opslaan

In [ ]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)